# Goodreads Data Exploration

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import gzip
import requests
import io
import os
from pyspark.sql import SparkSession
import pyspark.pandas as ps

/Users/SeanDuffy/opt/anaconda3/lib/python3.8/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
# Spark Data Loading

#spark = SparkSession.builder.getOrCreate()
#books_df.head()
#books_df = spark.read.json('data/goodreads_books.json')

In [38]:
books_df = pd.read_json(gzip.open('data/goodreads_books.json.gz'), lines=True, chunksize=10000)
authors_df = pd.read_json(gzip.open('data/goodreads_book_authors.json.gz'), lines=True, chunksize=10000)
genres_df = pd.read_json(gzip.open('data/goodreads_book_genres_initial.json.gz'), lines=True, chunksize=10000)
reviews_df = pd.read_json(gzip.open('data/goodreads_reviews_spoiler_raw.json.gz'), lines=True, chunksize=10000)
interactions_df = pd.read_csv('data/goodreads_interactions.csv', chunksize=10000)

In [39]:
for chunk in books_df:
    books_chunk = chunk
    break

for chunk in authors_df:
    authors_chunk = chunk
    break

for chunk in genres_df:
    genres_chunk = chunk

for chunk in reviews_df:
    reviews_chunk = chunk
    break

for chunk in interactions_df:
    int_chunk = chunk
    break

In [40]:
books_chunk = books_chunk.drop(columns=['series', 'asin', 'kindle_asin', 'similar_books', 'link', 'url', 'image_url',
                                       'edition_information', 'title_without_series', 'popular_shelves', 'publisher'])

def extract_authors(authors_dict):
    return [author['author_id'] for author in authors_dict]
    
authors_column = books_chunk['authors'].apply(extract_authors)
books_chunk['author_id'] = authors_column
books_chunk = books_chunk.explode('author_id').set_index('book_id')
books_chunk['author_id'] = books_chunk['author_id'].fillna(0).astype('int64')
books_chunk = books_chunk.join(authors_chunk, how='inner', on='author_id', lsuffix='_book', rsuffix='_author')

genres_chunk = genres_chunk['genres'].apply(pd.Series).join(genres_chunk)
genres_chunk = genres_chunk.drop(columns=['genres']).fillna(0)
genres_chunk = genres_chunk.set_index('book_id')
books_chunk = books_chunk.join(genres_chunk, how='inner', on='book_id')

In [51]:
reviews_chunk['review_text'] = reviews_chunk['review_text'].apply(lambda x: x.split(' '))

In [59]:
reviews_chunk.explode('review_text').groupby(by='review_text').count().sort_values('book_id', ascending=False)[251:301]

,user_id,book_id,review_id,rating,date_added,date_updated,read_at,started_at,n_votes,n_comments
review_text,,,,,,,,,,
keep,911,911,911,911,911,911,911,911,911,911
us,909,909,909,909,909,909,909,909,909,909
second,908,908,908,908,908,908,908,908,908,908
review,902,902,902,902,902,902,902,902,902,902
wants,898,898,898,898,898,898,898,898,898,898
finally,896,896,896,896,896,896,896,896,896,896
My,891,891,891,891,891,891,891,891,891,891
away,890,890,890,890,890,890,890,890,890,890
absolutely,889,889,889,889,889,889,889,889,889,889


In [60]:
reviews_chunk.explode('review_text').groupby(by='review_text').count().sort_values('book_id', ascending=False)

,user_id,book_id,review_id,rating,date_added,date_updated,read_at,started_at,n_votes,n_comments
review_text,,,,,,,,,,
the,78725,78725,78725,78725,78725,78725,78725,78725,78725,78725
and,62650,62650,62650,62650,62650,62650,62650,62650,62650,62650
I,51414,51414,51414,51414,51414,51414,51414,51414,51414,51414
to,50190,50190,50190,50190,50190,50190,50190,50190,50190,50190
a,46837,46837,46837,46837,46837,46837,46837,46837,46837,46837
...,...,...,...,...,...,...,...,...,...,...
ancients??,1,1,1,1,1,1,1,1,1,1
and--learning,1,1,1,1,1,1,1,1,1,1
and--schedule,1,1,1,1,1,1,1,1,1,1


In [63]:
int_chunk.drop(columns=['is_read', 'is_reviewed']).groupby(by='book_id').mean()

,user_id,rating
book_id,,
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
12706,14.0,5.0
12707,14.0,5.0
12708,14.0,5.0


In [64]:
reviews_chunk.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,"[This, is, a, special, book., It, started, slo...",Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,"[Recommended, by, Don, Katz., Avail, for, free...",Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,,,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"[A, fun,, fast, paced, science, fiction, thril...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,"[Recommended, reading, to, understand, what, i...",Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,,,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"[I, really, enjoyed, this, book,, and, there, ...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1
